# HIPT TCGA BRCA Dataset Curation

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
from IPython.display import Markdown as md

pd.set_option('mode.chained_assignment', None)

### Load HIPT slides

In [ ]:
hipt_dir = Path('/data/pathology/projects/ais-cap/code/git/opensource/HIPT')

In [ ]:
fold_0_fp = Path(hipt_dir, '2-Weakly-Supervised-Subtyping/splits/10foldcv_subtype/tcga_brca/splits_0.csv')

In [ ]:
df = pd.read_csv(fold_0_fp, index_col=0)
df.head()

In [ ]:
hipt_train_brca = list(df['train'].dropna().unique())
hipt_tune_brca = list(df['val'].dropna().unique())
hipt_test_brca = list(df['test'].dropna().unique())

len(hipt_train_brca), len(hipt_tune_brca), len(hipt_test_brca)

In [ ]:
hipt_train_brca = set(hipt_train_brca)
hipt_tune_brca = set(hipt_tune_brca)
hipt_test_brca = set(hipt_test_brca)

In [ ]:
intersection_train_tune = hipt_train_brca.intersection(hipt_tune_brca)
intersection_train_test = hipt_train_brca.intersection(hipt_test_brca)
intersection_tune_test = hipt_tune_brca.intersection(hipt_test_brca)

len(intersection_train_tune), len(intersection_train_test), len(intersection_tune_test)

In [ ]:
hipt_brca = hipt_train_brca | hipt_tune_brca | hipt_test_brca
len(hipt_brca)

In [ ]:
list(hipt_brca)[0]

### Load Blissey slides

In [ ]:
data_dir = Path('/data/pathology/archives/breast/TCGA_diagnostics/')

In [ ]:
blissey_brca = list(Path(data_dir, 'images').glob('*.tif')) + list(Path(data_dir, 'clement_slides/tif').glob('*.tif'))
blissey_brca = [s.stem for s in blissey_brca]
blissey_brca[0]

In [ ]:
len(blissey_brca)

In [ ]:
blissey_brca = set(blissey_brca)
len(blissey_brca)

### Compare HIPT & Blissey slides

In [ ]:
intersection_hipt_blissey = hipt_brca.intersection(blissey_brca)
print(f'{len(intersection_hipt_blissey)}/{len(hipt_brca)}') 

In [ ]:
difference_hipt_blissey = hipt_brca.difference(blissey_brca)
md(f'there are **{len(difference_hipt_blissey)}** slides from HIPT dataset that are not on Blissey')

### Load HIPT labels

In [ ]:
hipt_labels_csv_path = Path(hipt_dir, '2-Weakly-Supervised-Subtyping/dataset_csv/tcga_brca_subset.csv.zip')
hipt_labels_df = pd.read_csv(hipt_labels_csv_path, index_col=0)
hipt_labels_df.head()

In [ ]:
len(hipt_labels_df)

In [ ]:
hipt_labels_df['slide_id'].dropna().nunique()

why do we only have labels for **937** slides?

In [ ]:
# drop .tif from slide ids
hipt_labels_df['slide_id'] = hipt_labels_df['slide_id'].apply(lambda x: Path(x).stem)

In [ ]:
hipt_labels_df.oncotree_code.value_counts(dropna=False)

In [ ]:
hipt_fold_labels_df = hipt_labels_df[hipt_labels_df['slide_id'].isin(hipt_brca)]
hipt_fold_labels_df.slide_id.dropna().nunique()

among the 937 slides for which we have labels, only **875** are used for training/tuning/testing<br>
this is because we drop all labels that are not IDC or ILC

In [ ]:
hipt_fold_labels_df.oncotree_code.value_counts(dropna=False)

In [ ]:
def map_otc_to_int(otc: str, missing_label: int = -1):
    if otc == 'IDC':
        return 0
    elif otc == 'ILC':
        return 1
    else:
        return missing_label

In [ ]:
hipt_fold_labels_df['label'] = hipt_fold_labels_df['oncotree_code'].apply(map_otc_to_int)

### Get HIPT slide count per case_id

In [ ]:
list(hipt_brca)[0]

In [ ]:
def extract_case_id_from_slide_id(slide_id):
    return '-'.join(slide_id.split('-')[:3])

extract_case_id_from_slide_id(list(hipt_brca)[0])

In [ ]:
hipt_brca_case_ids = [extract_case_id_from_slide_id(s) for s in hipt_brca]
len(hipt_brca_case_ids), len(set(hipt_brca_case_ids))

Hence, we have some slides mapped to the same case ids

In [ ]:
duplicated_hipt_brca_case_ids = set([c for c in hipt_brca_case_ids if hipt_brca_case_ids.count(c) > 1])
md(f'there are **{len(duplicated_hipt_brca_case_ids)}** case ids mapped to 2+ slides')

In [ ]:
d = {'case_id': [], 'nslide': []}
for case_id in set(hipt_brca_case_ids):
    d['case_id'].append(case_id)
    d['nslide'].append(hipt_brca_case_ids.count(case_id))
tmp = pd.DataFrame.from_dict(d)
tmp.head()

In [ ]:
tmp.nslide.sum()

In [ ]:
ax = sns.countplot(data=tmp, x='nslide')
ax.bar_label(ax.containers[0], padding=5)
plt.xlabel('# slide')
plt.ylabel('# case')
plt.ylim(0,1000)
plt.show()

Investigate labels when multiple slides available

In [ ]:
tmp = hipt_fold_labels_df[hipt_fold_labels_df['case_id'].isin(duplicated_hipt_brca_case_ids)]
tmp.oncotree_code.value_counts(dropna=False)

question: is there a case_id whith 2+ slides whose labels are different?

In [ ]:
tmp_df = (tmp.groupby('case_id')['oncotree_code'].nunique() > 1).reset_index()
tmp_df = tmp_df.rename(columns={'oncotree_code': 'multiple_labels'})
tmp_df.head()

In [ ]:
tmp = tmp_df[tmp_df['multiple_labels'] == True]
len(set(tmp.case_id))

answer is **no**<br>
to extract some information from this, we'd need to know how these labels were actually curated<br>
e.g. whether patient/case level label was replicated for each slide (when 2+ slides available)

### Wrapping up

In [ ]:
hipt_fold_labels_df.case_id.dropna().nunique(), hipt_fold_labels_df.slide_id.dropna().nunique()

if one sticks to their _labeled_ dataset: **837** patients mapped to **875** slides

In [ ]:
# how many of these 875 slides are missing on the cluster?

hipt_brca_labeled = set(hipt_fold_labels_df.slide_id.dropna().tolist())
len(hipt_brca_labeled)

In [ ]:
intersection_hipt_labeled_blissey = hipt_brca_labeled.intersection(blissey_brca)
print(f'{len(intersection_hipt_labeled_blissey)}/{len(hipt_brca_labeled)}')

From HIPT github repo ([see here](https://github.com/mahmoodlab/HIPT/issues/6#issuecomment-1175792504)):

> Files under `HIPT/2-Weakly-Supervised-Subtyping/dataset_csv/` contain the total list of WSIs evaluated. Though there are more patients included in the `10foldcv_subtype/tcga_brca/split_{i}.csv` split, patients with insufficient tissue content for patching at the 4K-level and are thus missing in the pt_files-type folder, get excluded / masked out when slicing the dataframe. I should update the split csv files to clarify this confusion.

Issues Watchlist:

- https://github.com/mahmoodlab/HIPT/issues/11 has GDC manifest text files
- https://github.com/mahmoodlab/HIPT/issues/16 question regarding BRCA subtyping labels

Dump slide filenames in .txt file

In [ ]:
list(hipt_brca_labeled)[0]

In [ ]:
with open(f'../tcga_brca.txt', 'w') as f:
    for slide in hipt_brca_labeled:
        f.write(f'{slide}.tif\n')